In [1]:
import requests
import re
from bs4 import BeautifulSoup
from time import sleep
import pandas as pd

In [2]:
url= "https://www.imdb.com/chart/top"
ac="text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9"
headers={"Referer":"https://www.google.com","Connection":"Keep-Alive","Accept-Language":"en-US,en;q=0.9","Accept-Encoding":"gzip, deflate, br","Accept":ac,"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36"}

In [3]:
html = requests.get(url,headers=headers)

In [4]:
soup = BeautifulSoup(html.content,'lxml')

In [5]:
movies_container = soup.find("tbody", {"class":"lister-list"})
each_movie_group =movies_container.find_all("tr")

In [6]:
movies_link = []
for movies in each_movie_group:
    movies_link.append(movies.find("a")['href'])

In [7]:
def get_imdb_rating(soup):
    
    imdb_rating= soup.find("div", {"class":"sc-e457ee34-2"}).find("span",{"class":"sc-e457ee34-1"}).text
    return imdb_rating

def get_popularity(soup):
    popularity = soup.find("div",{"class":"sc-8bc809cd-1"}).text
    return popularity

def get_movie_genre(soup):
    movies_genre_html= soup.find_all("span",{"class":"ipc-chip__text"})
    movie_list= [i.text for i in movies_genre_html if("Back" not in  i.text)]
    return movie_list

def get_review_info(soup):
    review = soup.find("div",{"class":"bwUSkO"}).next_sibling
    review_info={}
    for list_item in review.find_all("li"):
        review_info[list_item.find("span",{"class":"label"}).text] = list_item.find("span",{"class":"score"}).text
    return review_info

def get_director(soup):
    director=soup.find("a",{"class":"ipc-metadata-list-item__list-content-item ipc-metadata-list-item__list-content-item--link"}).text
    return director

def get_writers(soup):
    writer_html = soup.find_all("li",{"class":"ipc-metadata-list__item"})
    writers = []
    for item in writer_html:
        if "Writers" in item.text:
            for links in item.find_all("a"):
                if links.text and links.text != "Writers":
                    writers.append(links.text)
            break
    return writers

def get_country_language_date(soup):
    detail_html = soup.find("section",{"data-testid": "Details"}).find("div",{"data-testid": "title-details-section"})
    detail = {}
    try:
        for item in detail_html.find_all("li"):
            if (item.div and item.button):
                if "Country" in item.button.text:
                    detail["country"] = item.div.text
                elif "Language" in item.button.text:
                    detail["language"] = item.div.text

        for item in detail_html.find_all("li"):
            if "date" in item.text:
                detail["release_date"]= item.a.next_sibling.text
                break
    except Exception as e:
        return None
    return detail

def get_budget_gross(soup):
    budget_html = soup.find("section",{"data-testid":"BoxOffice"})
    budget_gross = {}
    for item in budget_html.find_all("li"):
        if item.button:
            if "Budget" in item.button.text:
                budget_gross["budget"]= item.button.next_sibling.text
            elif "Gross worldwide" in item.button.text:
                budget_gross["gross"]= item.button.next_sibling.text

    return budget_gross

def get_duration(soup):
    duration_html = soup.find("section",{"data-testid":"TechSpecs"})
    duration = None
    for item in duration_html.find_all("li"):
        if item.button and "Runtime" in item.button:
            duration= item.button.next_sibling.text
    return duration

In [8]:
final_list= []
for ind, link in enumerate(movies_link):   
    movie_page = requests.get("https://www.imdb.com"+link,headers= headers)
    soup = BeautifulSoup(movie_page.content,'lxml')
    try:
        title = soup.h1.text
        popularity= get_popularity(soup)
        genre = get_movie_genre(soup)
        director = get_director(soup)
        imdb_rating = get_imdb_rating(soup)
        budget = get_budget_gross(soup)["budget"]
        gross = get_budget_gross(soup)["gross"]
        language = get_country_language_date(soup)["language"]
        released_date= get_country_language_date(soup)["release_date"]
        duration = get_duration(soup)
        writers = get_writers(soup)
        user_review = get_review_info(soup)["User reviews"]
        critic_review=  get_review_info(soup)["Critic reviews"]
        metascore=  get_review_info(soup)["Metascore"]
    except Exception as e:
        print(ind+1,e)
        continue
    
    final_list.append({
        "rank": ind+1,
        "title":title,
        "popularity":popularity,
        "genre":",".join(genre),
        "director":director,
        "imdb_rating":imdb_rating,
        "budget":budget,
        "gross":gross,
        "language":language,
        "released_date":released_date,
        "duration":duration,
        "writers":",".join(writers),
        "user_review":user_review,
        "critic_review":critic_review,
        "metascore":metascore
    })
    sleep(1)

45 'budget'
52 'NoneType' object has no attribute 'text'
59 'Metascore'
62 'Metascore'
86 'budget'
93 'budget'
96 'budget'
99 'budget'
107 'NoneType' object has no attribute 'find_all'
121 'NoneType' object has no attribute 'text'
122 'budget'
125 'NoneType' object has no attribute 'text'
128 'NoneType' object has no attribute 'text'
145 'NoneType' object has no attribute 'text'
175 'NoneType' object has no attribute 'text'
177 'NoneType' object has no attribute 'text'
179 'gross'
182 'gross'
186 'NoneType' object has no attribute 'text'
187 'NoneType' object has no attribute 'text'
188 'budget'
193 'NoneType' object has no attribute 'text'
195 'NoneType' object has no attribute 'text'
197 'NoneType' object has no attribute 'text'
199 'Metascore'
207 'NoneType' object has no attribute 'text'
208 'NoneType' object has no attribute 'text'
223 'NoneType' object has no attribute 'text'
227 'NoneType' object has no attribute 'text'
229 'Metascore'
231 'NoneType' object has no attribute 'tex

In [12]:
df = pd.DataFrame(final_list)

In [16]:
df.to_csv("movie_rating.csv",index=False)